# 주택단위 분석을 위한 자료 및 코드 set-up
- 주택 특성마다 가격이 다르게 형성 >> 이런 연구를 어떻게 하는지에 대한 조사(오피스텔과 아파트의 특성 타입 비교)

In [115]:
import pandas as pd
import numpy as np
import json


%matplotlib inline
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=12)
mpl.rc('xtick', labelsize=8)
mpl.rc('ytick', labelsize=8)


matplotlib.rc('font', family='AppleGothic')  # MacOS
plt.rcParams['axes.unicode_minus'] = False


# pandas max row 
pd.options.display.max_rows = 1000


## 아파트

- 전세
- 월세
- 매매

위 세개를 같이 놓고 비교 해야한다. → 트렌드 양상 비교 

`공식` = 전세금 * 시중금리(그해의) = 월세 (반대도 가능)




In [116]:
apt_trd = pd.read_csv('data/3.세종시_아파트(매매)_실거래가.csv')
apt_mth_yr = pd.read_csv('data/7.세종시_아파트(전월세)_실거래가.csv')

In [117]:
apt_trd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18409 entries, 0 to 18408
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   계약년월      18409 non-null  int64  
 1   계약일       18409 non-null  int64  
 2   시군구       18409 non-null  object 
 3   본번        18409 non-null  object 
 4   부번        18409 non-null  int64  
 5   단지명       18409 non-null  object 
 6   전용면적(㎡)   18409 non-null  float64
 7   거래금액(만원)  18409 non-null  object 
 8   층         18409 non-null  int64  
 9   건축년도      18409 non-null  int64  
 10  해제사유발생일   490 non-null    float64
dtypes: float64(2), int64(5), object(4)
memory usage: 1.5+ MB


In [118]:
apt_trd.head(3)

,계약년월,계약일,시군구,본번,부번,단지명,전용면적(㎡),거래금액(만원),층,건축년도,해제사유발생일
0,201701,19,세종특별자치시 고운동,1401,0,가락마을13단지(대광로제비앙),59.6550,"23,000",11,2015,NaN
1,201701,2,세종특별자치시 고운동,1393,0,가락마을15단지(중흥파크뷰),84.9894,"32,000",3,2015,NaN
2,201701,14,세종특별자치시 고운동,1395,0,가락마을16단지(에듀그린),84.8788,"32,500",21,2015,NaN


In [119]:
apt_mth_yr.head(3)

,계약년월,계약일,시군구,본번,부번,단지명,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,건축년도
0,201701,13,세종특별자치시 고운동,1396,0,가락마을 20단지(호반베르디움5차),전세,84.9895,"10,000",0,13,2015
1,201701,14,세종특별자치시 고운동,1396,0,가락마을 20단지(호반베르디움5차),전세,84.9895,"18,000",0,19,2015
2,201701,14,세종특별자치시 고운동,1396,0,가락마을 20단지(호반베르디움5차),전세,59.8338,"17,000",0,10,2015


In [120]:
# 전세와 월세 분할
apt_yr = apt_mth_yr[apt_mth_yr['전월세구분']=='전세'].drop(['전월세구분', '월세(만원)'], axis=1)
apt_mth = apt_mth_yr[apt_mth_yr['전월세구분']=='월세'].drop('전월세구분', axis=1)

In [121]:
apt_yr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26935 entries, 0 to 38893
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   계약년월     26935 non-null  int64  
 1   계약일      26935 non-null  int64  
 2   시군구      26935 non-null  object 
 3   본번       26935 non-null  object 
 4   부번       26935 non-null  int64  
 5   단지명      26935 non-null  object 
 6   전용면적(㎡)  26935 non-null  float64
 7   보증금(만원)  26935 non-null  object 
 8   층        26935 non-null  int64  
 9   건축년도     26935 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 2.3+ MB


In [122]:
apt_mth.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11960 entries, 4 to 38894
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   계약년월     11960 non-null  int64  
 1   계약일      11960 non-null  int64  
 2   시군구      11960 non-null  object 
 3   본번       11960 non-null  object 
 4   부번       11960 non-null  int64  
 5   단지명      11960 non-null  object 
 6   전용면적(㎡)  11960 non-null  float64
 7   보증금(만원)  11960 non-null  object 
 8   월세(만원)   11960 non-null  int64  
 9   층        11960 non-null  int64  
 10  건축년도     11960 non-null  int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 1.1+ MB


In [123]:
apt_trd['시군구'].str.split(' ').str[1].value_counts()

조치원읍    3583
종촌동     2790
아름동     2329
고운동     1984
한솔동     1919
도담동     1589
새롬동      634
다정동      592
소담동      577
금남면      432
어진동      422
보람동      380
나성동      290
연동면      178
대평동      167
연서면      129
전의면      104
반곡동      103
부강면       93
장군면       71
소정면       21
전동면       21
가람동        1
Name: 시군구, dtype: int64

In [124]:
def processData(df, price_column):
    # 법정동 구분
    df['법정동'] = df['시군구'].str.split(' ').str[1]

    # 숫자형 변환 
    df[price_column] = df[price_column].str.replace(',','').astype('int64')


    # 문자열 변환
    df['계약년월'] = df['계약년월'].astype(str)
    df['계약일'] = df['계약일'].astype(str)


    # datetime 변환
    df['계약년월일'] = pd.to_datetime(df['계약년월'] + df['계약일'], format='%Y%m%d')
    df = df.drop(columns=['계약일'])
# df['건축년도'] = df['건축년도'].dt.to_period('Y')
    return df



In [125]:
apt_trd = processData(apt_trd, '거래금액(만원)')
apt_yr = processData(apt_yr, '보증금(만원)')
apt_mth= processData(apt_mth, '보증금(만원)')



## 아파트 매매

In [126]:
apt_trd.head()

,계약년월,시군구,본번,부번,단지명,전용면적(㎡),거래금액(만원),층,건축년도,해제사유발생일,법정동,계약년월일
0,201701,세종특별자치시 고운동,1401,0,가락마을13단지(대광로제비앙),59.6550,23000,11,2015,NaN,고운동,2017-01-19
1,201701,세종특별자치시 고운동,1393,0,가락마을15단지(중흥파크뷰),84.9894,32000,3,2015,NaN,고운동,2017-01-02
2,201701,세종특별자치시 고운동,1395,0,가락마을16단지(에듀그린),84.8788,32500,21,2015,NaN,고운동,2017-01-14
3,201701,세종특별자치시 고운동,1386,0,가락마을17단지(골드클래스),59.9159,25100,7,2015,NaN,고운동,2017-01-09
4,201701,세종특별자치시 고운동,1390,0,가락마을21단지(에듀포레),99.9434,32250,3,2015,NaN,고운동,2017-01-09


## 아파트 전세

In [127]:
apt_yr.head(3)

,계약년월,시군구,본번,부번,단지명,전용면적(㎡),보증금(만원),층,건축년도,법정동,계약년월일
0,201701,세종특별자치시 고운동,1396,0,가락마을 20단지(호반베르디움5차),84.9895,10000,13,2015,고운동,2017-01-13
1,201701,세종특별자치시 고운동,1396,0,가락마을 20단지(호반베르디움5차),84.9895,18000,19,2015,고운동,2017-01-14
2,201701,세종특별자치시 고운동,1396,0,가락마을 20단지(호반베르디움5차),59.8338,17000,10,2015,고운동,2017-01-14


## 아파트 월세

In [128]:
apt_mth.head(3)

,계약년월,시군구,본번,부번,단지명,전용면적(㎡),보증금(만원),월세(만원),층,건축년도,법정동,계약년월일
4,201702,세종특별자치시 고운동,1396,0,가락마을 20단지(호반베르디움5차),84.9895,10000,30,13,2015,고운동,2017-02-04
12,201702,세종특별자치시 고운동,1396,0,가락마을 20단지(호반베르디움5차),59.8338,4000,38,7,2015,고운동,2017-02-28
32,201706,세종특별자치시 고운동,1396,0,가락마을 20단지(호반베르디움5차),59.8338,1000,46,17,2015,고운동,2017-06-19


## Merge 매매, 전세, 월세 

In [129]:
apt_trd = apt_trd.rename(columns={'거래금액(만원)':'거래금액'})
apt_trd['유형'] = '매매'

In [130]:
# 전세
apt_yr = apt_yr.rename(columns={'보증금(만원)':'거래금액'})
apt_yr['유형'] = '전세'

# 월세
apt_mth['유형'] = '월세'

In [131]:
apt_mth.head(3)

,계약년월,시군구,본번,부번,단지명,전용면적(㎡),보증금(만원),월세(만원),층,건축년도,법정동,계약년월일,유형
4,201702,세종특별자치시 고운동,1396,0,가락마을 20단지(호반베르디움5차),84.9895,10000,30,13,2015,고운동,2017-02-04,월세
12,201702,세종특별자치시 고운동,1396,0,가락마을 20단지(호반베르디움5차),59.8338,4000,38,7,2015,고운동,2017-02-28,월세
32,201706,세종특별자치시 고운동,1396,0,가락마을 20단지(호반베르디움5차),59.8338,1000,46,17,2015,고운동,2017-06-19,월세


### 월세 -> 전세
- 전월세전환률: 4.4	

In [132]:
apt_mth['거래금액'] = apt_mth['월세(만원)']*12 / 0.044 + apt_mth['보증금(만원)']
apt_mth = apt_mth.drop(columns=['보증금(만원)',	'월세(만원)'])

In [133]:
apt_yr_mth =  apt_yr.append(apt_mth)
apt = apt_trd.append(apt_yr_mth)

In [138]:
apt

,계약년월,시군구,본번,부번,단지명,전용면적(㎡),거래금액,층,건축년도,해제사유발생일,법정동,계약년월일,유형
0,201701,세종특별자치시 고운동,1401,0,가락마을13단지(대광로제비앙),59.6550,23000.000000,11,2015,NaN,고운동,2017-01-19,매매
1,201701,세종특별자치시 고운동,1393,0,가락마을15단지(중흥파크뷰),84.9894,32000.000000,3,2015,NaN,고운동,2017-01-02,매매
2,201701,세종특별자치시 고운동,1395,0,가락마을16단지(에듀그린),84.8788,32500.000000,21,2015,NaN,고운동,2017-01-14,매매
3,201701,세종특별자치시 고운동,1386,0,가락마을17단지(골드클래스),59.9159,25100.000000,7,2015,NaN,고운동,2017-01-09,매매
4,201701,세종특별자치시 고운동,1390,0,가락마을21단지(에듀포레),99.9434,32250.000000,3,2015,NaN,고운동,2017-01-09,매매
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38880,202010,세종특별자치시 해밀동,6402,1,해밀마을 2단지,84.9500,27090.909091,2,2020,NaN,해밀동,2020-10-13,월세
38884,202010,세종특별자치시 해밀동,6402,1,해밀마을 2단지,84.9500,24818.181818,10,2020,NaN,해밀동,2020-10-28,월세
38890,202011,세종특별자치시 해밀동,6402,1,해밀마을 2단지,102.8300,37272.727273,5,2020,NaN,해밀동,2020-11-21,월세
38891,202012,세종특별자치시 해밀동,6402,1,해밀마을 2단지,102.5300,35909.090909,14,2020,NaN,해밀동,2020-12-08,월세


In [162]:
apt['계약년월']

0        201701
1        201701
2        201701
3        201701
4        201701
          ...  
38880    202010
38884    202010
38890    202011
38891    202012
38894    202012
Name: 계약년월, Length: 57304, dtype: object